In [1]:
from os import environ
from tempfile import mkdtemp

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', mkdtemp())

TOOL_ID = 'ONSSET'
output_folder


'/tmp/tmp4relvhkv'

In [2]:
import pandas as pd
from os.path import join, splitext

specifications_path = join(output_folder, 'specifications.xlsx')
json_output_path = join(output_folder, 'assets.json')

excel_writer = pd.ExcelWriter(specifications_path)
for file_name in [
    'specs_data.csv',
    'scenario_info.csv',
    'scenario_parameters.csv',
]:
    table = pd.read_csv(join(input_folder, file_name))
    base_name = splitext(file_name)[0]
    sheet_name = ''.join(_.title() for _ in base_name.split('_'))
    table.to_excel(excel_writer, sheet_name=sheet_name, index=False)
excel_writer.save()

In [3]:
from os import makedirs
from os.path import join

details_folder = join(output_folder, 'details')
summaries_folder = join(output_folder, 'summaries')
try:
    makedirs(details_folder)
    makedirs(summaries_folder)
except OSError:
    pass

In [4]:
from onsset.runner import calibration, scenario

cities_path = join(input_folder, 'cities.csv')
calibrated_specifications_path = join(output_folder, 'specifications.xlsx')
calibrated_cities_path = join(output_folder, 'cities.csv') 

calibration(
    specs_path=specifications_path,
    csv_path=cities_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path,
    results_folder=details_folder,
    summary_folder=summaries_folder)

2021-06-16 13:46:08,890		Populate ResidentialDemandTier columns
2021-06-16 13:46:08,892		Ensure that columns that are supposed to be numeric are numeric
2021-06-16 13:46:08,909		Replace null values with zero
2021-06-16 13:46:08,911		Sort by country, Y and X
2021-06-16 13:46:08,915		Classify road dist
2021-06-16 13:46:08,921		Classify substation dist
2021-06-16 13:46:08,925		Classify elevation
2021-06-16 13:46:08,928		Classify slope
2021-06-16 13:46:08,932		Classify land cover
2021-06-16 13:46:08,935		Combined classification
2021-06-16 13:46:08,939		Grid penalty
2021-06-16 13:46:08,943		Calculate Wind CF
2021-06-16 13:46:08,984		Population calibration process
2021-06-16 13:46:09,009		Urban/rural calibration process
2021-06-16 13:46:09,121		Population projection process


The calibrated population differs by 0.00. In case this is not acceptable please revise this part of the code


2021-06-16 13:46:09,259		Calibrate current electrification
2021-06-16 13:46:09,360		Calibration finished. Results are transferred to the csv file


We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
The modelled electrification rate differ by 0.00. Urban elec. rate differ by 0.00 and Rural elec. rate differ by 0.00. 
If this is not acceptable please revise this part of the algorithm
Djibouti
Scenario: 1


2021-06-16 13:46:09,590		Determine current MV line length
2021-06-16 13:46:09,595		Calculate new connections
2021-06-16 13:46:09,603		Setting electrification demand as per target per year
2021-06-16 13:46:09,640		Calculate minigrid hydro LCOE
2021-06-16 13:46:09,691		Calculate minigrid PV LCOE
2021-06-16 13:46:09,743		Calculate minigrid wind LCOE
2021-06-16 13:46:09,794		Calculate standalone PV LCOE
2021-06-16 13:46:09,823		Determine minimum technology (off-grid)
2021-06-16 13:46:09,846		Determine minimum tech LCOE
2021-06-16 13:46:09,861		Define the initial electrification status
2021-06-16 13:46:09,983		Initially 245 electrified
2021-06-16 13:46:10,112		Electrification loop 1 with 245 electrified
2021-06-16 13:46:10,215		Calculate investment cost
2021-06-16 13:46:10,223		Determine electrification limits
2021-06-16 13:46:10,235		Determine final electrification decision
2021-06-16 13:46:10,238		Calculate new capacity
2021-06-16 13:46:10,250		Calculate summaries
2021-06-16 13:46:10,312	

The electrification rate achieved in 2025 is 83.3 %


2021-06-16 13:46:10,458		Calculate minigrid wind LCOE
2021-06-16 13:46:10,516		Calculate standalone PV LCOE
2021-06-16 13:46:10,544		Determine minimum technology (off-grid)
2021-06-16 13:46:10,569		Determine minimum tech LCOE
2021-06-16 13:46:10,584		Define the initial electrification status
2021-06-16 13:46:10,690		Initially 245 electrified
2021-06-16 13:46:10,806		Electrification loop 1 with 453 electrified
2021-06-16 13:46:10,870		Electrification loop 2 with 49 electrified
2021-06-16 13:46:10,942		Electrification loop 3 with 8 electrified
2021-06-16 13:46:11,016		Electrification loop 4 with 1 electrified
2021-06-16 13:46:11,044		Calculate investment cost
2021-06-16 13:46:11,054		Determine electrification limits
2021-06-16 13:46:11,058		Determine final electrification decision
2021-06-16 13:46:11,060		Calculate new capacity
2021-06-16 13:46:11,075		Calculate summaries


The electrification rate achieved in 2030 is 100.0 %


2021-06-16 13:46:11,349		Finished


In [12]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    scenarios = pd.concat(tables)
    scenarios.to_csv(target_path, index=False)
    return scenarios
    
scenarios = collect_scenarios(join(output_folder, 'details.csv'), details_folder)
summaries = collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)

report_year = summaries.columns[-1]
summaries

,ScenarioName,Unnamed: 0,2025,2030
0,dj-1-1_1_1_1_0_0_summary,1.Population_Grid,9.009693e+05,1.111083e+06
1,dj-1-1_1_1_1_0_0_summary,1.Population_SA_Diesel,0.000000e+00,0.000000e+00
2,dj-1-1_1_1_1_0_0_summary,1.Population_SA_PV,0.000000e+00,3.546339e+04
3,dj-1-1_1_1_1_0_0_summary,1.Population_MG_Diesel,0.000000e+00,0.000000e+00
4,dj-1-1_1_1_1_0_0_summary,1.Population_MG_PV,4.308232e+03,3.195504e+04
5,dj-1-1_1_1_1_0_0_summary,1.Population_MG_Wind,0.000000e+00,6.488908e+02
6,dj-1-1_1_1_1_0_0_summary,1.Population_MG_Hydro,0.000000e+00,0.000000e+00
7,dj-1-1_1_1_1_0_0_summary,1.Population_MG_Hybrid,0.000000e+00,0.000000e+00
8,dj-1-1_1_1_1_0_0_summary,2.New_Connections_Grid,3.183693e+05,2.058052e+05
9,dj-1-1_1_1_1_0_0_summary,2.New_Connections_SA_Diesel,0.000000e+00,0.000000e+00
